In [1]:
import numpy as np 
import matplotlib.pyplot as plt 

def get_sin_cos(theta):
    return (np.cos(theta), np.sin(theta))

def rad2deg(rad):
    return np.array(rad) * (180.0/np.pi)

def deg2rad(deg):
    return np.array(deg) * (np.pi/180.0)

def rotationX(theta):
    sin_theta, cos_theta = get_sin_cos(theta)
    return np.array([[1,0,0,],
                    [0,  cos_theta, sin_theta ],
                    [0, -sin_theta, cos_theta ]])

def rotationY(theta):
    sin_theta, cos_theta = get_sin_cos(theta)
    return np.array([[cos_theta,0, -sin_theta],
                    [0, 1, 0 ],
                    [sin_theta, 0, cos_theta ]])

def rotationZ(theta):
    sin_theta, cos_theta = get_sin_cos(theta)
    return np.array([[cos_theta , sin_theta, 0  ],
                    [-sin_theta , cos_theta, 0  ],
                    [0, 0, 1 ]])

def printQuaternion(title, quat):
    print(f"Quaternion {title} [{quat[0]},  {quat[1]},  {quat[2]}, {quat[3]}]")



In [ ]:
def quatNorm(quat):
    q0 = quat[0]
    q1 = quat[1]
    q2 = quat[2]
    q3 = quat[3]
    return np.sqrt(q0 * q0 + q1 * q1 + q2 * q2 + q3 * q3)

def conjugate(quat):
    q0 = quat[0]
    q1 = quat[1]
    q2 = quat[2]
    q3 = quat[3]
    return np.array([q0,-q1,-q2,-q3])

def quaternionInv(quat):
    conj = conjugate(quat)
    norm = norm(quatNorm)
    return conj/norm

def quaternionMult(quat1, quat2):
    q0 = quat1[0]
    q1 = quat1[1]
    q2 = quat1[2]
    q3 = quat1[3]
    Q = np.array([[q0, -q1, -q2, -q3],
              [q1,  q0,  q3, -q2 ],
              [q2, -q3,  q0, q1 ],
              [q3,  q2, -q1, q0 ]])
    return np.matmul(Q, quat2)